<a href="https://colab.research.google.com/github/HarukiKozukapenguin/FixedHydrus_imitation_navigation/blob/main/Project%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import drive
import pandas as pd
import csv
import random
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import math
import random
import glob

drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import csv
import random
from scipy.spatial import KDTree
import matplotlib.pyplot as plt

class Node:
    """
    Node class for dijkstra search
    """

    def __init__(self, x, y, yaw, cost, parent_index):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.cost = cost
        self.parent_index = parent_index

    def __str__(self):
        return str(self.x) + "," + str(self.y) + ","+ str(self.yaw) + "," +\
               str(self.cost) + "," + str(self.parent_index)

# 自作エラー,pathが見つからなかった場合に使用
class InitCollisionError(Exception):
    pass
class GoalCollisionError(Exception):
    pass
class PathNotFoundError(Exception):
    pass

class Prm:
    def __init__(self, robot_theta: list, edge_l: float, fly_area: list, tree_csv: str, file_num: int, x_add_range:float, y_add_range:float, act_norm: float):
        self.N_KNN = 10  # number of edge from one sampled point
        self.MAX_EDGE_LEN = 5.0  # [m] Maximum edge length
        self.yaw_weight = 2.0
        self.show_animation = True
        self.edge_l = edge_l
        self.act_norm = act_norm
        self.change_edge = True

        theta: list = [np.deg2rad(i) for i in robot_theta] # from deg to rad
        bR2: np.array = np.array([0.0,0.0])
        bC2 = np.array([-edge_l/2,0.0])
        bC2ToR1 = np.array([-edge_l/2*np.cos(theta[0]),edge_l/2*np.sin(theta[0])])
        bR1 = bC2+bC2ToR1
        bC1 = bC2+bC2ToR1*2
        bC3 = np.array([edge_l/2,0.0])
        bC3ToR3 = np.array([edge_l/2*np.cos(theta[1]),edge_l/2*np.sin(theta[1])])
        bR3 = bC3+bC3ToR3
        bC4 = bC3+bC3ToR3*2
        bC4ToR4 = np.array([edge_l/2*np.cos(theta[1]+theta[2]),edge_l/2*np.sin(theta[1]+theta[2])])
        bR4 = bC4+bC4ToR4
        bC5 = bC4+bC4ToR4*2
        CG = (bR1+bR2+bR3+bR4)/4
        C_list = [bC1-CG,bC2-CG,bC3-CG,bC4-CG,bC5-CG]
        mid_point = []
        for idx in range(len(C_list)-1):
            mid_point.append((C_list[idx] + C_list[idx+1])/2)
        collision_list = C_list + mid_point

        self.C_list = [pos.reshape(2, 1) for pos in C_list]
        self.collision_list = [pos.reshape(2, 1) for pos in collision_list]

        # print("self.C_list: ",self.C_list)

        self.min_area, self.max_area = fly_area
        for i in range(2):
            if self.min_area[i]>=self.max_area[i]:
                print("[ERROR] min_area>max_area")

        path = tree_csv+"/environment_" + str(file_num) + "/static_obstacles.csv"
        obstacle_x_list = []
        obstacle_y_list = []
        self.radius_list = []
        with open(path) as f:
            reader = csv.reader(f)
            for row in reader:
                obstacle_x_list.append(float(row[1]))
                obstacle_y_list.append(float(row[2]))
                self.radius_list.append(float(row[8]))
        self.obstacle_kd_tree = KDTree(np.vstack((obstacle_x_list, obstacle_y_list)).T)
        
        self.add_range=[]
        self.add_range.append(x_add_range)
        self.add_range.append(y_add_range)
        
    def calc(self, init_pos: list, goal_pos: list, node_num:int):
        # cample points
        self.init_pos = init_pos
        self.goal_pos = goal_pos
        self.sample_points(init_pos, goal_pos, node_num)
       
        # show animation:
        # color = [i for i in range(len(self.node_list))]
        # sample_x = [node[0] for node in self.node_list]
        # sample_y = [node[1] for node in self.node_list]
        # fig = plt.figure(figsize = (10, 10))
        # ax = fig.add_subplot(1,1,1)
        # ax.scatter(sample_x, sample_y)
        # for i in range(len(sample_x)):
        #     plt.text(sample_x[i], sample_y[i],str(i))
        # plt.show()

        # sample_yaw = [node[2] for node in self.node_list]
        # fig = plt.figure(figsize = (10, 10))
        # ax = fig.add_subplot(1,1,1)
        # ax.scatter(sample_x, sample_yaw)
        # for i in range(len(sample_x)):
        #     plt.text(sample_x[i], sample_yaw[i],str(i))
        # plt.show()
    
    def collide(self, pos: list):
        CoG_pos = np.array([pos[0],pos[1]])
        CoG_pos.resize((2,1))
        yaw = pos[2]
        R = np.array([
            [np.cos(yaw),-np.sin(yaw)],
            [np.sin(yaw),np.cos(yaw)]
            ])
        for apex in self.collision_list:
            apex = np.array(apex)
            apex.resize((2,1))
            # print("R.shape: ",R.shape)
            # print("apex.shape: ",apex.shape)
            t = CoG_pos+R@apex
            tx = t[0,0]
            ty = t[1,0]
            dist, index = self.obstacle_kd_tree.query([tx, ty])
            if len(self.radius_list)!=0 and dist <= self.radius_list[index]:
                return True
        return False
    
    def sample_points(self, init_pos: list, goal_pos: list, node_num:int):
        min_range=[]
        max_range=[]
        for i in range(2):
            if not ((self.min_area[i]<init_pos[i]-self.add_range[i]) and (self.min_area[i]<goal_pos[i]-self.add_range[i])):
                print("min is out of range")
                min_range.append(self.min_area[i])
            else:
                min_range.append(min(init_pos[i]-self.add_range[i],goal_pos[i]-self.add_range[i]))
            if not ((init_pos[i]+self.add_range[i]<self.max_area[i]) and (goal_pos[i]+self.add_range[i]<self.max_area[i])):
                print("max is out of range")
                max_range.append(self.max_area[i])
            else:
                max_range.append(max(init_pos[i]+self.add_range[i],goal_pos[i]+self.add_range[i]))
        self.node_list = []
        for _ in range(node_num):
            node_pos = [min_range[i]+(max_range[i]-min_range[i])*random.random() for i in range(2)]
            node_pos.append(random.uniform(-np.pi, np.pi))
            if not self.collide(node_pos):
                self.node_list.append(node_pos)

        if not self.collide(init_pos):
            self.node_list.append(init_pos)
        else:
            raise InitCollisionError("init_pos is collide")
        
        if not self.collide(goal_pos):
            self.node_list.append(goal_pos)
        else:
            raise GoalCollisionError("goal_pos is collide")
    
    def generate_road_map(self, rr=0): #rr is radius of robot
        self.road_map = []
        sample_x = [node[0] for node in self.node_list]
        sample_y = [node[1] for node in self.node_list]
        sample_yaw = [node[2]*self.yaw_weight for node in self.node_list] 

        n_sample = len(sample_x) #this num is different from node_num in calc, for this eliminates collide candidates
        # Be careful, we didn
        sample_kd_tree = KDTree(np.vstack((sample_x, sample_y,sample_yaw)).T) # this kd_tree is for connection between nodes


        for (i, ix, iy,iyaw) in zip(range(n_sample), sample_x, sample_y,sample_yaw): #select one point from sample points

            dists, indexes = sample_kd_tree.query([ix, iy,iyaw], k=n_sample) #sort from selected sample point
            edge_id = []
            true_iyaw = iyaw/self.yaw_weight #iyawは重み付けされたyawなので、実際のyawとは異なるため、正しいyawに戻す

            for ii in range(1, len(indexes)): #nodes around given node
                nx = sample_x[indexes[ii]]
                ny = sample_y[indexes[ii]]
                nyaw = sample_yaw[indexes[ii]]
                true_nyaw = nyaw/self.yaw_weight

                if not self.edge_collision(ix, iy, true_iyaw, nx, ny, true_nyaw, rr):
                    edge_id.append(indexes[ii])

                if len(edge_id) >= self.N_KNN:
                    break
            self.road_map.append(edge_id)

    def edge_collision(self, sx, sy, syaw, gx, gy, gyaw, rr):
        x = sx
        y = sy
        yaw = syaw
        dx = gx - sx
        dy = gy - sy
        dyaw = gyaw - syaw
        direction = np.array([dx, dy,dyaw])
        d = np.linalg.norm(direction)
        if d >= self.MAX_EDGE_LEN:
            return True
        direction = direction / d
        # math.atan2(gy - sy, gx - sx)
        D = max(rr,self.edge_l/2)
        n_step = int(d / D) # n_step is the number of steps to reach goal point round down

        for i in range(n_step):
            pos = [x,y,yaw]
            if self.collide(pos):
                return True # collision
            x += D * direction[0]
            y += D * direction[1]
            yaw += D * direction[2]
        pos = [gx,gy,gyaw]
        if self.collide(pos):
            return True
        return False  # OK

    def edge_collision_deb(self, sx, sy, syaw, gx, gy, gyaw, rr):
        x = sx
        y = sy
        yaw = syaw
        dx = gx - sx
        dy = gy - sy
        dyaw = gyaw - syaw
        direction = np.array([dx, dy,dyaw])
        d = np.linalg.norm(direction)
        if d >= self.MAX_EDGE_LEN:
            return True
        direction = direction / d
        # math.atan2(gy - sy, gx - sx)
        D = max(rr,self.edge_l/2)
        n_step = int(d / D) # n_step is the number of steps to reach goal point round down
        print("n_step",n_step)

        self.pos_deb = []

        for i in range(n_step):
            pos = [x,y,yaw]
            print("i is",i)
            print("pos is ",pos)
            self.pos_deb.append(pos)
            if self.collide(pos):
                return True # collision
            x += D * direction[0]
            y += D * direction[1]
            yaw += D * direction[2]
        pos = [gx,gy,gyaw]
        print("pos is ",pos)
        self.pos_deb.append(pos)
        if self.collide(pos):
            return True
        return False  # OK

    def dijkstra_planning(self) ->tuple:
        """
        s_x: start x position [m]
        s_y: start y position [m]
        s_yaw: start yaw angle [rad]
        goal_x: goal x position [m]
        goal_y: goal y position [m]
        goal_yaw: goal yaw angle [rad]
        @return: Two lists of path coordinates ([x1, x2, ...], [y1, y2, ...],[yaw1, yaw2, ...]), empty list when no path was found
        """

        start_node = Node(self.init_pos[0],self.init_pos[1],self.init_pos[2], 0.0, -1)
        goal_node = Node(self.goal_pos[0],self.goal_pos[1], self.goal_pos[2], 0.0, -1)

        open_set, closed_set = dict(), dict()
        open_set[len(self.road_map) - 2] = start_node

        path_found = True

        while True:
            if not open_set:
                raise PathNotFoundError("Cannot find path")
                path_found = False
                break

            c_id = min(open_set, key=lambda o: open_set[o].cost)
            current = open_set[c_id]

            # show graph
            # if self.show_animation and len(closed_set.keys()) % 2 == 0:
            #     # for stopping simulation with the esc key.
            #     plt.gcf().canvas.mpl_connect(
            #         'key_release_event',
            #         lambda event: [exit(0) if event.key == 'escape' else None])
            #     plt.plot(current.x, current.y,current.yaw, "xg")
            #     plt.pause(0.001)

            if c_id == (len(self.road_map) - 1):
                # print("goal is found!")
                goal_node.parent_index = current.parent_index
                goal_node.cost = current.cost
                break

            # Remove the item from the open set
            del open_set[c_id]
            # Add it to the closed set
            closed_set[c_id] = current

            # expand search grid based on motion model
            for i in range(len(self.road_map[c_id])):
                n_id = self.road_map[c_id][i]
                dx = self.node_list[n_id][0] - current.x
                dy = self.node_list[n_id][1] - current.y
                dyaw = self.node_list[n_id][2] - current.yaw
                d = math.hypot(dx, dy,dyaw)
                node = Node(self.node_list[n_id][0], self.node_list[n_id][1], self.node_list[n_id][2],
                            current.cost + d, c_id)

                if n_id in closed_set:
                    continue
                # Otherwise if it is already in the open set
                if n_id in open_set:
                    if open_set[n_id].cost > node.cost:
                        open_set[n_id].cost = node.cost
                        open_set[n_id].parent_index = c_id
                else:
                    open_set[n_id] = node

        # generate final course
        rx, ry, ryaw = [goal_node.x], [goal_node.y], [goal_node.yaw]
        parent_index = goal_node.parent_index
        while parent_index != -1:
            n = closed_set[parent_index]
            rx.append(n.x)
            ry.append(n.y)
            ryaw.append(n.yaw)
            parent_index = n.parent_index

        return rx, ry, ryaw
    
    def prm_planning(self, init_pos: list, goal_pos: list, node_num: int):
        self.calc(init_pos=init_pos,goal_pos=goal_pos, node_num=node_num)
        self.generate_road_map()
        self.path_output = self.dijkstra_planning()
        self.current_edge = len(self.path_output[0])-1 #最後のedgeから計算
    
    def forward(self, current_pos: list) -> np.array:
        # calculate direction
        if self.change_edge:
            # print("current edge is ",self.current_edge)
            direction = [self.path_output[0][self.current_edge-1] - self.path_output[0][self.current_edge], self.path_output[1][self.current_edge-1] - self.path_output[1][self.current_edge], self.path_output[2][self.current_edge-1] - self.path_output[2][self.current_edge]]
            # print("edge's direction is ",direction)
            # print("edge's beginning is ",[self.path_output[0][self.current_edge],self.path_output[1][self.current_edge],self.path_output[2][self.current_edge]])
            # print("edge's end is ",[self.path_output[0][self.current_edge-1],self.path_output[1][self.current_edge-1],self.path_output[2][self.current_edge-1]])
            
            direction = np.array([direction[0], direction[1], direction[2]*self.yaw_weight])
            direction_norm = np.linalg.norm(direction)
            # print("direction_norm is ",direction_norm)
            self.edge_left = direction_norm
            direction = direction/direction_norm
            # direction = np.array([direction[0], direction[1], direction[2]/self.yaw_weight])
            self.change_edge = False
            self.act = direction * self.act_norm
            self.act = np.array([self.act[0], self.act[1], self.act[2]/self.yaw_weight])
            # print("not last point's act is ",self.act)
        else:
            self.edge_left -= self.act_norm

        if self.edge_left < self.act_norm:
            # print("last point of edge")
            # print("current_pos is",current_pos)
            # print("self.edge_left is ",self.edge_left)
            self.change_edge = True

            direction = [self.path_output[0][self.current_edge-1] - current_pos[0], self.path_output[1][self.current_edge-1] - current_pos[1], self.path_output[2][self.current_edge-1] - current_pos[2]]
            # print("act in last edge is ", direction)

            self.act = np.array(direction)
            if self.act_norm<np.linalg.norm(np.array([self.act[0],self.act[1],self.act[2]*self.yaw_weight])):
                print("self.act is larger than self.act_norm")
            self.current_edge -= 1
        return self.act

# if __name__ == "__main__":
#     prm = Prm(robot_theta=[0,60,0],edge_l=0.6,fly_area=[[2,-5.0],[12,5.0]],tree_csv="/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder",file_num=499,x_add_range=2.0,y_add_range=2.0, act_norm = 0.1)
#     path = prm.prm_planning(init_pos=[5.0,-1,0.1],goal_pos=[9.0,0,0], node_num=100)
#     # prm.generate_road_map()
#     print(prm.road_map)
#     # prm.dijkstra_planning()

PRMが問題なくうごているのか確認したところ、pathの接続がうまくいっていないことが判明


バグはiyawなどの値がself.yaw_weight倍された値のまま衝突判定がされていたことが原因

In [ ]:
import matplotlib.collections as mc
from google.colab import files
figure, axes = plt.subplots()

def visualize(given_pos: list):
    path = tree_csv+"/environment_" + str(file_num) + "/static_obstacles.csv"
    with open(path) as f:
        reader = csv.reader(f)
        for row in reader:
            x = float(row[1])
            y = float(row[2])
            r = float(row[8]) #+body_size
            draw_circle = plt.Circle((x, y), r)
            axes.add_artist(draw_circle)

    CoG_pos = np.array([
    [given_pos[0]],
    [given_pos[1]]
    ])
    yaw = given_pos[2]
    r = np.array([
    [np.cos(yaw), -np.sin(yaw)],
    [np.sin(yaw), np.cos(yaw)]
        ])
    # print("prm.C_list: ",prm.C_list)
    edge_list = [CoG_pos + r @ pos for pos in prm.C_list]
    # print("edge_list: ",edge_list)
    lines = [[(edge_list[i][0,0],edge_list[i][1,0]),(edge_list[i+1][0,0],edge_list[i+1][1,0])]
    for i in range(len(edge_list)-1)]
    lc = mc.LineCollection(lines, colors = "g",linewidths=2)
    axes.add_collection(lc)
    print("fig_num: " , fig_num)
    plt.savefig(str(fig_num))
    files.download("temp_figure")


# plt.figure(figsize=(8,8))
plt.xlim([-0.3, 12])
plt.ylim([-5.0, 5.0])
tree_csv = "/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
file_num = 500
fig_num:int = 0
# global fig_num
print("out of fig_num: " , fig_num)

# prmでの点での状態を確認
x_list = prm.path_output[0]
y_list = prm.path_output[1]
yaw_list = prm.path_output[2]

for x,y,yaw in zip(x_list,y_list,yaw_list):
    visualize([x,y,yaw])
# prm.path_output #明らかに障害物に衝突しそうなedgeが含まれている
# idx = 3
# prm.edge_collision_deb(prm.path_output[0][idx], prm.path_output[1][idx], prm.path_output[2][idx], prm.path_output[0][idx-1], prm.path_output[1][idx-1], prm.path_output[2][idx-1],0)
# # True(collisionしている)ということになっているが、なぜかedgeに含まれている
# prm.pos_deb

In [ ]:
import matplotlib.collections as mc
from google.colab import files
import matplotlib.pyplot as plt
figure, axes = plt.subplots()
tree_csv = "/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
plt.xlim([-2, 2])
plt.ylim([-2, 2])
path = tree_csv+"/environment_" + str(file_num) + "/static_obstacles.csv"
CoG_pos = np.array([
    [0],
    [0]
    ])
yaw = 0
r = np.array([
[np.cos(yaw), -np.sin(yaw)],
[np.sin(yaw), np.cos(yaw)]
    ])
# print("prm.C_list: ",prm.C_list)
edge_list = [CoG_pos + r @ pos for pos in prm.C_list]
# print("edge_list: ",edge_list)
lines = [[(edge_list[i][0,0],edge_list[i][1,0]),(edge_list[i+1][0,0],edge_list[i+1][1,0])]
for i in range(len(edge_list)-1)]
lc = mc.LineCollection(lines, colors = "g",linewidths=2)
axes.add_collection(lc)
print("fig_num: " , fig_num)
plt.savefig(str(fig_num))
# files.download("temp_figure")

In [ ]:
class Cylinder:
    max_dist = 10 #class variable
    def __init__(self, x, y, radius):
        self.x = x
        self.y = y
        self.radius = radius

    def calculate_depth(self, x, y, theta):
        # 円柱との交点を計算する関数
        def intersect(cx, cy, radius, x, y, theta)->float:
            # (x, y)を原点とした座標系において、円柱の中心(cx, cy)を移動させる
            cx -= x
            cy -= y
            # theta方向に移動させる
            c, s = math.cos(-theta), math.sin(-theta)
            cx, cy = cx * c - cy * s, cx * s + cy * c
            # 交点を計算する
            # a = cx**2 + cy**2
            # b = 2 * (cx * x + cy * y)
            # c = x**2 + y**2 - radius**2
            # discriminant = b**2 - 4*a*c
            discriminant = radius**2-cy**2
            if discriminant < 0:
                # 交点がない場合は、無限遠のdepthを返す
                return self.max_dist
            # t1 = (-b - math.sqrt(discriminant)) / (2 * a)
            # # t1が0より小さい場合は、infを出力
            t = cx - np.sqrt(discriminant)
            if t < 0:
                return self.max_dist
            # t = t1
            return t

        # 交点方向の長さを計算
        return intersect(self.x, self.y, self.radius, x, y, theta)

    @classmethod
    def calculate_depths(cls, cylinders: list, x, y, thetas:list)->list:
        # thetasの要素ごとにdepthを計算する
        if len(cylinders)==0:
            return [cls.max_dist for theta in thetas]
        else:
            return [min([c.calculate_depth(x, y, theta) for c in cylinders]) for theta in thetas]
class Env:
    def __init__(self, robot_theta: list, edge_l: float, fly_area: list, tree_csv: str, file_num: int, init_pos: list,goal_pos: list, thetas: list):
        self.yaw_weight = 2.0
        self.goal_determine = 0.2
        theta = [np.deg2rad(i) for i in robot_theta] # from deg to rad
        bR2 = np.array([0.0,0.0])
        bC2 = np.array([-edge_l/2,0.0])
        bC2ToR1 = np.array([-edge_l/2*np.cos(theta[0]),edge_l/2*np.sin(theta[0])])
        bR1 = bC2+bC2ToR1
        bC1 = bC2+bC2ToR1*2
        bC3 = np.array([edge_l/2,0.0])
        bC3ToR3 = np.array([edge_l/2*np.cos(theta[1]),edge_l/2*np.sin(theta[1])])
        bR3 = bC3+bC3ToR3
        bC4 = bC3+bC3ToR3*2
        bC4ToR4 = np.array([edge_l/2*np.cos(theta[1]+theta[2]),edge_l/2*np.sin(theta[1]+theta[2])])
        bR4 = bC4+bC4ToR4
        bC5 = bC4+bC4ToR4*2
        CG = (bR1+bR2+bR3+bR4)/4
        C_list = [bC1-CG,bC2-CG,bC3-CG,bC4-CG,bC5-CG]
        self.C_list = [pos.reshape(2, 1) for pos in C_list]

        self.thetas : list = [-theta for theta in reversed(thetas)] + thetas
        
        # print("self.C_list: ",self.C_list)

        self.min_area, self.max_area = fly_area
        for i in range(2):
            if self.min_area[i]>=self.max_area[i]:
                print("[ERROR] min_area>max_area")

        path = tree_csv+"/environment_" + str(file_num) + "/static_obstacles.csv"
        self.obstacle_x_list = []
        self.obstacle_y_list = []
        self.radius_list = []
        with open(path) as f:
            reader = csv.reader(f)
            self.cylinders = [Cylinder(float(row[1]),float(row[2]),float(row[8])) for row in reader]
            # for row in reader:
            #     self.obstacle_x_list.append(float(row[1]))
            #     self.obstacle_y_list.append(float(row[2]))
            #     self.radius_list.append(float(row[8]))
        self.pos: np.array = np.array(init_pos)
        self.goal_pos = np.array(goal_pos)
        self.done = False

    def step(self, action: np.array):
        self.pos[0]+=action[0]
        self.pos[1]+=action[1]
        self.pos[2]+=action[2]
        obs = self.calc_observation(self.pos)
        self.done = self.is_terminate() #terminate data collection and begin to set new position, delete environment
        return np.concatenate([self.pos, obs])

    def calc_observation(self, pos: np.array):
        depths:list = Cylinder.calculate_depths(self.cylinders, pos[0], pos[1], self.thetas)
        return np.array(depths)
    
    def is_terminate(self):
        check_pos = np.array([self.pos[0],self.pos[1],self.pos[2]*self.yaw_weight])
        check_goal = np.array([self.goal_pos[0],self.goal_pos[1],self.goal_pos[2]*self.yaw_weight])
        return np.linalg.norm(check_pos-check_goal)<self.goal_determine

In [ ]:
# debug: edgeの位置を変更するときにself.act_norm以上のactが出てきてしまっているため、変更
if __name__ == "__main__":
    # hyper parameters
    robot_theta = [0,60,0]
    edge_l = 0.6
    fly_area=[[0,-5.0],[60,5.0]]
    tree_csv="/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
    x_add_range=2.0
    y_add_range=2.0
    thetas = [5,15,25,35,45,60,75,90,105,120,135,150,165]
    thetas = [np.deg2rad(theta) for theta in thetas]
    act_norm = 0.4
    state_data = np.empty((0,3+len(thetas)*2), float)
    act_data = np.empty((0,3),float)

    CONNECTION_RETRY = 8

    file_num = 500
    # environment making
    init_pos=[5.0,-1,0]
    goal_pos=[10.0,0,0]
    node_num=100
    sample_data=True
    for i in range(1, CONNECTION_RETRY + 1):
        try:
            prm = Prm(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area ,tree_csv=tree_csv, file_num=file_num, x_add_range=x_add_range, y_add_range=y_add_range, act_norm = act_norm)
            prm.prm_planning(init_pos=init_pos, goal_pos=goal_pos, node_num=node_num)
        except InitCollisionError as e:
            print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
            print("error type is ",str(type(e)))
            init_pos = [pos + random.uniform(-0.3, 0.3) for pos in init_pos]
            goal_pos = [pos + random.uniform(-0.3, 0.3) for pos in goal_pos]
            if i==CONNECTION_RETRY:
                sample_data=False
        except GoalCollisionError as e:
            print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
            print("error type is ",str(type(e)))
            init_pos = [pos + random.uniform(-0.3, 0.3) for pos in init_pos]
            goal_pos = [pos + random.uniform(-0.3, 0.3) for pos in goal_pos]
            if i==CONNECTION_RETRY:
                sample_data=False
        except PathNotFoundError as e:
            print("In", file_num, "path is not found. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
            print("error type is ",str(type(e)))
            node_num+=100
            if i==CONNECTION_RETRY:
                sample_data=False
        else:
            break
    env = Env(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area, tree_csv=tree_csv, file_num=file_num, init_pos=init_pos, goal_pos=goal_pos, thetas=thetas)

    # running environment and collecting data
    pos = init_pos
    act = prm.forward(pos)
    # print("prm.current_edge is :",prm.current_edge)
    new_obs = env.step(act)

    while not env.is_terminate():
        # state_data = np.hstack([state_data, new_obs])
        state_data = np.append(state_data, new_obs.reshape(1,-1), axis=0)
        act = prm.forward(pos)
        act_data = np.append(act_data, act.reshape(1,-1), axis=0)
        new_obs = env.step(act)
        pos = env.pos
    print(file_num," is done")
    np.savetxt("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/state_data/"+"case_"+str(file_num)+".csv", state_data, delimiter=',', fmt='%.5e')
    np.savetxt("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/act_data/"+"case_"+str(file_num)+'.csv', act_data, delimiter=',', fmt='%.5e')
        
    # 最初はダミーデータが入るため、消去
    state_data = state_data[1:]
    act_data = act_data[1:]

In [ ]:
prm.path_output

In [ ]:
non_last_act = np.array( [0.36680185, 0.06338815, 0.14641838])
edge_direction = np.array([1.3053784807749391, 0.2255864491762989, 0.5210753372568608])
edge_beginning = np.array( [5.046473912148123, -1.3813197721636434, 0.14773693680920624])
current_pos = edge_beginning
for _ in range(6):
    # edge_direction-=non_last_act
    current_pos += non_last_act
    # print("edge_direction",edge_direction)
    print("current_pos",current_pos)

In [ ]:
# visualize start to goal for debug
figure, axes = plt.subplots()


# plt.figure(figsize=(8,8))
plt.xlim([-0.3, 12])
plt.ylim([-5.0, 5.0])
tree_csv = "/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
file_num = 500
fig_num:int = 0
# global fig_num
print("out of fig_num: " , fig_num)

for data in state_data[::5]:
    visualize([data[0],data[1],data[2]])
    fig_num+=5


# data = state_data[0]
# visualize([data[0],data[1],data[2]])
# fig_num+=1

# visualize([7.970126625404299,-0.895266442631526,-0.008050072924890461])
# visualize([5.0,-1,0])
# visualize(prm.node_list[36])
# It should be connected between 43 and 84 in this situation...

In [ ]:
prm.path_output

In [ ]:
# 一つの環境で環境を入手して、そのstepでの学習結果の取得を行う
if __name__ == "__main__":
    # hyper parameters
    robot_theta = [0,60,0]
    edge_l = 0.6
    fly_area=[[0,-5.0],[60,5.0]]
    tree_csv="/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
    x_add_range=2.0
    y_add_range=2.0
    thetas = [5,15,25,35,45,60,75,90,105,120,135,150,165]
    thetas = [np.deg2rad(theta) for theta in thetas]
    act_norm = 1.5

    CONNECTION_RETRY = 8

    for file_num in reversed(range(501)):
        state_data = np.empty((0,3+len(thetas)*2), float)
        act_data = np.empty((0,3),float)
        # reset state_data, act_data when changeing different envirioment
        # environment making
        init_pos=[5.0,-1,0]
        goal_pos=[55.0,0,0]
        node_num=100
        sample_data=True
        for i in range(1, CONNECTION_RETRY + 1):
            try:
                prm = Prm(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area ,tree_csv=tree_csv, file_num=file_num, x_add_range=x_add_range, y_add_range=y_add_range, act_norm = act_norm)
                prm.prm_planning(init_pos=init_pos, goal_pos=goal_pos, node_num=node_num)
            except InitCollisionError as e:
                print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
                print("error type is ",str(type(e)))
                init_pos = [pos + random.uniform(-0.3, 0.3) for pos in init_pos]
                if i==CONNECTION_RETRY:
                    sample_data=False
            except GoalCollisionError as e:
                print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
                print("error type is ",str(type(e)))
                goal_pos = [pos + random.uniform(-0.3, 0.3) for pos in goal_pos]
                if i==CONNECTION_RETRY:
                    sample_data=False
            except PathNotFoundError as e:
                print("In", file_num, "path is not found. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
                print("error type is ",str(type(e)))
                node_num+=100
                if i==CONNECTION_RETRY:
                    sample_data=False
            else:
                break
        if not sample_data:
            continue
        env = Env(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area, tree_csv=tree_csv, file_num=file_num, init_pos=init_pos, goal_pos=goal_pos, thetas=thetas)

        # running environment and collecting data
        pos = init_pos
        act = prm.forward(pos)
        # print("prm.current_edge is :",prm.current_edge)
        new_obs = env.step(act)

        while not env.is_terminate():
            # state_data = np.hstack([state_data, new_obs])
            state_data = np.append(state_data, new_obs.reshape(1,-1), axis=0)
            act = prm.forward(pos)
            act_data = np.append(act_data, act.reshape(1,-1), axis=0)
            new_obs = env.step(act)
            pos = env.pos
        print(file_num," is done")
        # 最初はダミーデータが入るため、消去
        state_data = state_data[1:]
        act_data = act_data[1:]
        np.savetxt("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/state_data/"+"case_"+str(file_num)+".csv", state_data, delimiter=',', fmt='%.5e')
        np.savetxt("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/act_data/"+"case_"+str(file_num)+'.csv', act_data, delimiter=',', fmt='%.5e')

In [ ]:
x = np.arange(8).reshape(1,-1)
print(x)
##[[0, 1, 2, 3],
##[4, 5, 6, 7]]

# y = np.arange(9, 17).reshape(2,4)
# print(y)
# ##[[ 9, 10, 11],
# ##[12, 13, 14]]

# x_add = np.append(x,y,axis=0)
# print(x_add)
# ##[[ 0,  1,  2,  3],
# ##[ 4,  5,  6,  7],
# ##[ 9, 10, 11, 12],
# ##[13, 14, 15, 16]]

In [ ]:
# 円柱を作成する
c1 = Cylinder(4, 2, 1)
c2 = Cylinder(2, 0, 1)
c3 = Cylinder(4, 0, 1)

# 円柱のリストを作成する
cylinders = [c1,c2,c3]

# 見たい位置のx座標、y座標、theta方向のリストを指定する
x = 2
y = 2
thetas = [0, math.pi / 2, math.pi, math.pi * 3 / 2]

# depthsを計算する
depths = Cylinder.calculate_depths(cylinders, x, y, thetas)
print(depths)

In [ ]:
str(fig_num)

In [ ]:
visualize([9.0,0,0])

In [ ]:
state_data

In [ ]:
ii=1
nx = sample_x[indexes[ii]]
ny = sample_y[indexes[ii]]
nyaw = sample_yaw[indexes[ii]]

print(prm.edge_collision(sample_x[-2], sample_y[-2], sample_yaw[-2], nx, ny, nyaw, 0)) #79と36はcollideしている

In [ ]:
CoG_pos = np.array([pos[0],pos[1]])
CoG_pos.resize((2,1))
yaw = pos[2]
R = np.array([
    [np.cos(yaw),-np.sin(yaw)],
    [np.sin(yaw),np.cos(yaw)]
    ])
for apex in self.C_list:
    apex = np.array(apex)
    apex.resize((2,1))
    # print("R.shape: ",R.shape)
    # print("apex.shape: ",apex.shape)
    t = CoG_pos+R@apex
    tx = t[0,0]
    ty = t[1,0]
    dist, index = self.obstacle_kd_tree.query([tx, ty])
    if len(self.radius_list)!=0 and dist <= self.radius_list[index]:
        return True

state_data,act_dataファイルをもとに学習を行う

In [ ]:
#作成したファイルを一つのcavファイルにまとめる

 
file_path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/state_data/"
csv_files = glob.glob(file_path + '/*')

for a in csv_files:
    print(a)

#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
    data_list.append(pd.read_csv(file,header=None))

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
df = pd.concat(data_list, axis=0,ignore_index=True, sort=True)

df.to_csv("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/state_data/whole_data.csv",index=False)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')

whole_data.csvファイルをもとにobsからactを出力させるデータを作成する
(本来であれば、trajの近さを考えて、学習を考えてみる必要もあるとは思った(cf. Deep-PANTHER Planner))

ただし、"Learning Perception-aware Agile Flight in Cluttered Environments"では、このような工夫がされずに学習が行われていた

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

# csvファイルをシャッフル
x_df = pd.read_csv("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/state_data/whole_data.csv",header=None)
y_df = pd.read_csv("/content/drive/MyDrive/先端人工知能論_プロジェクト/自作データ/act_data/whole_data.csv",header=None)
teacher_df = pd.concat([x_df, y_df], axis=1,ignore_index=True) #横結合(cf. https://datasciencemore.com/python-pandas-concat/)
teacher_df = teacher_df.sample(frac=1) #データシャッフル(cf. https://note.nkmk.me/python-pandas-random-sort-shuffle/)

# 教師データ,テストデータの分割
train, test = train_test_split(teacher_df, test_size=0.1)

# pandas to numpy data (cf. https://dreamer-uma.com/pytorch-dataset/)
x_train = torch.FloatTensor(train.loc[:,:3+len(thetas)*2-1].values)
y_train = torch.FloatTensor(train.loc[:,3+len(thetas)*2:].values)
x_test = torch.FloatTensor(test.loc[:,:3+len(thetas)*2-1].values)
y_test = torch.FloatTensor(test.loc[:,3+len(thetas)*2:].values)
# x_df, y_df　のデータ結合に変更

In [ ]:
len(x_df)

In [ ]:
x_train[:,:3]

In [ ]:
# ref: https://qiita.com/sudamasahiko/items/b54fed1ffe8bb6d48818 https://learn.microsoft.com/ja-jp/windows/ai/windows-ml/tutorials/pytorch-train-model
# ref: network 関係でsuper内で自身のnetworkの名前は省略可能 https://tzmi.hatenablog.com/entry/2019/12/31/002627

In [ ]:
# Todo
# 単純に全てのobsをNNに突っ込む

# Define a convolution neural network
batchsize = 100
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3+len(thetas)*2, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 3)

    def forward(self, input):
        output = F.relu(self.fc1(input))
        output = F.relu(self.fc2(output))    
        output = F.relu(self.fc3(output))                        
        output = F.relu(self.fc4(output))
        return output

# Instantiate a neural network model 
model = Network()

In [ ]:
loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

def saveModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/learned_model.pth"
    torch.save(model.state_dict(), path)

def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    test_n = x_train.shape[0] # test_num

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i in range(0, test_n, batchsize):
            x = x_train[i: i+batchsize]
            y = y_train[i: i+batchsize]
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(x)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, y)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i/batchsize % 100 == 99:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0
        saveModel()

if __name__ == "__main__": 
    # Let's build our model
    train(20)

In [ ]:
# obsの情報をNNで次元圧縮してから学習に突っ込む
class Vision_network(nn.Module):
    def __init__(self):
        super().__init__()
        self.obs1 = nn.Linear(len(thetas)*2, 256)
        self.obs2 = nn.Linear(256, 512)
        self.obs3 = nn.Linear(512, 512)
        self.obs4 = nn.Linear(512, 256)
        self.obs5 = nn.Linear(256, 128)
        self.obs6 = nn.Linear(128, 32)
        self.obs7 = nn.Linear(32, 8)
        self.fc1 = nn.Linear(3+8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 3)

    def forward(self, input):
        pos = input[:,:3]
        obs = input[:,3:]
        obs = F.relu(self.obs1(obs))
        obs = F.relu(self.obs2(obs))
        obs = F.relu(self.obs3(obs))
        obs = F.relu(self.obs4(obs))
        obs = F.relu(self.obs5(obs))
        obs = F.relu(self.obs6(obs))
        obs = F.relu(self.obs7(obs))
        output = torch.cat([pos, obs],axis=1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))    
        output = F.relu(self.fc3(output))                        
        output = F.relu(self.fc4(output))
        return output
model = Vision_network()

In [ ]:
loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
loss_calcularion_interval: int = 100 #how often you print running_loss
batchsize = 100

def saveModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/learned_model.pth"
    torch.save(model.state_dict(), path)

def train(num_epochs):
    
    # best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    test_n = x_train.shape[0] # test_num

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i in range(0, test_n, batchsize):
            x = x_train[i: i+batchsize]
            y = y_train[i: i+batchsize]
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(x)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, y)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss/loss_calcularion_interval))
                # zero the loss
                running_loss = 0.0
    saveModel()

if __name__ == "__main__": 
    # Let's build our model
    train(20)

In [ ]:
loss_calcularion_interval: int = 10

def eval():
    test_n = x_test.shape[0] # test_num
    running_loss = 0.0
    for i in range(0, test_n, batchsize):
        x = x_test[i: i+batchsize]
        y = y_test[i: i+batchsize]
        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(x)
        # compute the loss based on model output and real labels
        loss = loss_fn(outputs, y)
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
            # print every 1000 (twice per epoch) 
            print('[%5d] loss: %.5f' %
                    (i + 1, running_loss/loss_calcularion_interval))
            # zero the loss
            running_loss = 0.0

if __name__ == "__main__": 
    # Let's build our model
    eval()

1dim-CNNを使用して、obsの次元を落とす

In [ ]:
# この記事を参考にして、kernelを奇数にした https://blog.shikoan.com/pytorch-convtranspose2d/
flatten = nn.Flatten(1, -1)
class CNN_1dim(nn.Module):
    def __init__(self):
        super().__init__()
        self.obs1 = nn.Conv1d(1,8,3,stride=1)
        # shape: [channel, length_signal] = [1,len(thetas)*2 (= 26)] -> [8,24]
        self.obs2 = nn.Conv1d(8,32,2,stride=1)
        # [8,24] -> [32,23]
        self.obs3 = nn.Conv1d(32,128,3,stride=1)
        # [32,23] -> [128,21]
        self.obs4 = nn.Conv1d(128,128,3,stride=2)
        # [128,21] -> [128,10]
        self.obs5 = nn.Conv1d(128,32,3,stride=1)
        # [32,10] -> [32,8]
        self.obs6 = nn.Conv1d(32,8,3,stride=1)
        # [32,8] -> [8,6]
        self.obs7 = nn.Conv1d(8,2,3,stride=1)
        # [8,6] -> [2,4] = 8次元 
        self.fc1 = nn.Linear(3+8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 3)

    def forward(self, input):
        pos = input[:,:3]
        obs = input[:,3:]
        obs = obs.unsqueeze(1)
        obs = F.relu(self.obs1(obs))
        obs = F.relu(self.obs2(obs))
        obs = F.relu(self.obs3(obs))
        obs = F.relu(self.obs4(obs))
        obs = F.relu(self.obs5(obs))
        obs = F.relu(self.obs6(obs))
        obs = F.relu(self.obs7(obs))
        obs = obs.view(-1,  self.num_flat_features(obs))

        output = torch.cat([pos, obs],axis=1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))    
        output = F.relu(self.fc3(output))                        
        output = F.relu(self.fc4(output))
        return output
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
model = CNN_1dim()

In [ ]:
loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
loss_calcularion_interval: int = 100 #how often you print running_loss
batchsize = 100

def saveModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/learned_model.pth"
    torch.save(model.state_dict(), path)

def train(num_epochs):
    
    # best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    test_n = x_train.shape[0] # test_num

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i in range(0, test_n, batchsize):
            x = x_train[i: i+batchsize]
            y = y_train[i: i+batchsize]
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(x)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, y)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss/loss_calcularion_interval))
                # zero the loss
                running_loss = 0.0
    saveModel()

if __name__ == "__main__": 
    # Let's build our model
    train(20)

In [ ]:
loss_calcularion_interval: int = 10
eval()

In [ ]:
 x = x_train[0: 0+batchsize]
 x.shape

In [ ]:
x = x.unsqueeze(1)
x.shape

In [ ]:
flatten_all = nn.Flatten(1, -1)
flatten_all(x).shape

In [ ]:
m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
output.shape

In [ ]:
import math
def encode_dim(L_in,kernel_size,stride):
    return math.floor((L_in-1*(kernel_size-1)-1)/stride+1)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encode1 = nn.Conv1d(1,8,3,stride=1)
        # shape: [channel, length_signal] = [1,len(thetas)*2 (= 26)] -> [8,24]
        self.encode2 = nn.Conv1d(8,32,2,stride=1)
        # [8,24] -> [32,23]
        self.encode3 = nn.Conv1d(32,128,3,stride=1)
        # [32,23] -> [128,21]
        self.encode4 = nn.Conv1d(128,128,3,stride=2)
        # [128,21] -> [128,10]
        self.encode5 = nn.Conv1d(128,32,3,stride=1)
        # [128,10] -> [32,8]
        self.encode6 = nn.Conv1d(32,8,3,stride=1)
        # [32,8] -> [8,6]
        self.encode7 = nn.Conv1d(8,2,3,stride=1)
        # [8,6] -> [2,4] = 8次元 

    def forward(self, obs):
        obs = obs.unsqueeze(1)
        obs = F.relu(self.encode1(obs))
        obs = F.relu(self.encode2(obs))
        obs = F.relu(self.encode3(obs))
        obs = F.relu(self.encode4(obs))
        obs = F.relu(self.encode5(obs))
        obs = F.relu(self.encode6(obs))
        obs = F.relu(self.encode7(obs))
        # flattenしていない値をencoderの出力とした
        return obs

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.decode7 = nn.ConvTranspose1d(2,8, kernel_size = 3 ,stride=1)
        # shape: [channel, length_signal] = [2,4] -> [8,6]
        self.decode6 = nn.ConvTranspose1d(8,32,kernel_size = 3 ,stride=1)
        # [8,6] -> [32,8]
        self.decode5 = nn.ConvTranspose1d(32,128, kernel_size = 3 ,stride=1)
        # [32,8] -> [128,10]
        self.decode4 = nn.ConvTranspose1d(128,128, kernel_size = 4 ,stride=2)
        # [128,10] -> [128,22]
        self.decode3 = nn.ConvTranspose1d(128,16, kernel_size = 3 ,stride=1)
        # [128,22] -> [16,24]
        self.decode2 = nn.ConvTranspose1d(16,1, kernel_size = 3 ,stride=1)
        # [16,24] -> [1,26]

    def forward(self, obs):
        obs = F.relu(self.decode7(obs))
        obs = F.relu(self.decode6(obs))
        obs = F.relu(self.decode5(obs))
        obs = F.relu(self.decode4(obs))
        obs = F.relu(self.decode3(obs))
        obs = F.relu(self.decode2(obs))
        encode = obs.squeeze()
        return encode

# このサイトをもとに、AutoEncoder2クラスを作成https://atmarkit.itmedia.co.jp/ait/articles/2007/31/news025.html
class AutoEncoder2(nn.Module):
    def __init__(self, enc, dec):
        super().__init__()
        self.enc = enc
        self.dec = dec
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        return x

encoder = Encoder()
decoder = Decoder()
model = AutoEncoder2(encoder, decoder)

In [ ]:
model

In [ ]:
import math
def decode_dim(L_in,kernel_size,stride):
    return (L_in-1)*stride+kernel_size

In [ ]:
decode_dim(24,3,1)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
loss_calcularion_interval: int = 100 #how often you print running_loss
batchsize = 100

def saveModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/auto_encoder.pth"
    torch.save(model.state_dict(), path)

def train(num_epochs):
    
    # best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    test_n = x_train.shape[0] # test_num

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i in range(0, test_n, batchsize):
            x = x_train[i: i+batchsize]
            true_obs = x[:,3:]

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(true_obs)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, true_obs)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss/loss_calcularion_interval))
                # zero the loss
                running_loss = 0.0
    saveModel()

if __name__ == "__main__": 
    # Let's build our model
    train(15)

In [ ]:
loss_calcularion_interval: int = 10

def eval():
    test_n = x_test.shape[0] # test_num
    running_loss = 0.0
    for i in range(0, test_n, batchsize):
        x = x_train[i: i+batchsize]
        true_obs = x[:,3:]
        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(true_obs)
        # compute the loss based on model output and real labels
        loss = loss_fn(outputs, true_obs)
        # backpropagate the loss
        # loss.backward()
        # adjust parameters based on the calculated gradients
        # optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
            # print every 1000 (twice per epoch) 
            print('[%5d] loss: %.5f' %
                    (i + 1, running_loss/loss_calcularion_interval))
            # zero the loss
            running_loss = 0.0

if __name__ == "__main__": 
    # Let's build our model
    eval()
# Auto-encoderは学習がうまくいっていることが確認できた

In [ ]:
# load only encoder
def loadEncoder():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/auto_encoder.pth"
    encoder = Encoder()
    decoder = Decoder()
    model = AutoEncoder2(encoder, decoder)
    model.load_state_dict(torch.load(path))
    # modelのparameterの変更をなくすため
    # https://tips-memo.com/pytorch-parameters-stop
    for param in model.parameters():
        param.requires_grad = False 
    return model.enc
# encoder = loadEncoder()

class CNN_2dim(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = loadEncoder()
        self.fc1 = nn.Linear(3+8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 3)

    def forward(self, input):
        pos = input[:,:3]
        obs = input[:,3:]
        obs = self.encoder(obs)
        obs = obs.view(-1,  self.num_flat_features(obs))

        output = torch.cat([pos, obs],axis=1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))    
        output = F.relu(self.fc3(output))                        
        output = F.relu(self.fc4(output))
        return output
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
model = CNN_2dim()
oss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
loss_calcularion_interval: int = 100 #how often you print running_loss
batchsize = 100

def saveModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/learned_model.pth"
    torch.save(model.state_dict(), path)

def train(num_epochs):
    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    test_n = x_train.shape[0] # test_num

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i in range(0, test_n, batchsize):
            x = x_train[i: i+batchsize]
            y = y_train[i: i+batchsize]
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(x)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, y)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss/loss_calcularion_interval))
                # zero the loss
                running_loss = 0.0
    saveModel()

if __name__ == "__main__": 
    # Let's build our model
    train(20)

In [ ]:
loss_calcularion_interval: int = 5
def eval():
    test_n = x_test.shape[0] # test_num
    running_loss = 0.0
    for i in range(0, test_n, batchsize):
        x = x_test[i: i+batchsize]
        y = y_test[i: i+batchsize]
        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(x)
        # compute the loss based on model output and real labels
        loss = loss_fn(outputs, y)
        # backpropagate the loss
        # loss.backward()
        # # adjust parameters based on the calculated gradients
        # optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        if i/batchsize % loss_calcularion_interval == loss_calcularion_interval-1:    
            # print every 1000 (twice per epoch) 
            print('[%5d] loss: %.5f' %
                    (i + 1, running_loss/loss_calcularion_interval))
            # zero the loss
            running_loss = 0.0

if __name__ == "__main__": 
    # Let's build our model
    eval()

#学習結果検証

In [ ]:
def loadModel():
    path = "/content/drive/MyDrive/先端人工知能論_プロジェクト/learned_model.pth"
    model = CNN_2dim()
    model.load_state_dict(torch.load(path))
    # modelのparameterの変更をなくすため
    # https://tips-memo.com/pytorch-parameters-stop
    for param in model.parameters():
        param.requires_grad = False 
    return model


In [ ]:
if __name__ == "__main__":
    # hyper parameters
    robot_theta = [0,60,0]
    edge_l = 0.6
    fly_area=[[0,-5.0],[60,5.0]]
    tree_csv="/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
    x_add_range=2.0
    y_add_range=2.0
    thetas = [5,15,25,35,45,60,75,90,105,120,135,150,165]
    thetas = [np.deg2rad(theta) for theta in thetas]
    act_norm = 0.4
    state_data = np.empty((0,3+len(thetas)*2), float)
    act_data = np.empty((0,3),float)

    CONNECTION_RETRY = 8

    file_num = 500
    # environment making
    init_pos=[5.0,-1,0]
    goal_pos=[10.0,0,0]
    node_num=100
    sample_data=True
    for i in range(1, CONNECTION_RETRY + 1):
        try:
            prm = Prm(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area ,tree_csv=tree_csv, file_num=file_num, x_add_range=x_add_range, y_add_range=y_add_range, act_norm = act_norm)
            prm.prm_planning(init_pos=init_pos, goal_pos=goal_pos, node_num=node_num)
        except InitCollisionError as e:
            print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
            print("error type is ",str(type(e)))
            init_pos = [pos + random.uniform(-0.3, 0.3) for pos in init_pos]
            goal_pos = [pos + random.uniform(-0.3, 0.3) for pos in goal_pos]
            if i==CONNECTION_RETRY:
                sample_data=False
        except GoalCollisionError as e:
            print("In", file_num, " start or goal is collide. retry:{i}/{max}".format(i=i,max=CONNECTION_RETRY))
            print("error type is ",str(type(e)))
            init_pos = [pos + random.uniform(-0.3, 0.3) for pos in init_pos]
            goal_pos = [pos + random.uniform(-0.3, 0.3) for pos in goal_pos]
            if i==CONNECTION_RETRY:
                sample_data=False
        else:
            break
    env = Env(robot_theta = robot_theta, edge_l=edge_l, fly_area=fly_area, tree_csv=tree_csv, file_num=file_num, init_pos=init_pos, goal_pos=goal_pos, thetas=thetas)
    model = loadModel()

    # running environment and collecting data
    pos = init_pos
    act = prm.forward(pos)
    # print("prm.current_edge is :",prm.current_edge)
    new_obs = env.step(act)
    print("type(new_obs): ",type(new_obs))

    for _ in range(30):
        # state_data = np.hstack([state_data, new_obs])
        state_data = np.append(state_data, new_obs.reshape(1,-1), axis=0)
        new_obs = new_obs.reshape(1,-1)
        # numpy to pytorch tensor for model calculation (cf. https://tzmi.hatenablog.com/entry/2020/02/16/170928)
        new_obs = torch.from_numpy(new_obs.astype(np.float32)).clone()
        act = model(new_obs)
        act_data = np.append(act_data, act.reshape(1,-1), axis=0)
        new_obs = env.step(act.flatten())
        pos = env.pos
    state_data = state_data[1:]
    act_data = act_data[1:]

In [ ]:
len(state_data[:,0])

In [ ]:
import matplotlib.collections as mc
from google.colab import files
figure, axes = plt.subplots()
tree_csv = "/content/drive/MyDrive/先端人工知能論_プロジェクト/tree_folder"
file_num = 500
fig_num:int = 0
def visualize(given_pos: list):
    path = tree_csv+"/environment_" + str(file_num) + "/static_obstacles.csv"
    with open(path) as f:
        reader = csv.reader(f)
        for row in reader:
            x = float(row[1])
            y = float(row[2])
            r = float(row[8]) #+body_size
            draw_circle = plt.Circle((x, y), r)
            axes.add_artist(draw_circle)

    CoG_pos = np.array([
    [given_pos[0]],
    [given_pos[1]]
    ])
    yaw = given_pos[2]
    r = np.array([
    [np.cos(yaw), -np.sin(yaw)],
    [np.sin(yaw), np.cos(yaw)]
        ])
    # print("prm.C_list: ",prm.C_list)
    edge_list = [CoG_pos + r @ pos for pos in prm.C_list]
    # print("edge_list: ",edge_list)
    lines = [[(edge_list[i][0,0],edge_list[i][1,0]),(edge_list[i+1][0,0],edge_list[i+1][1,0])]
    for i in range(len(edge_list)-1)]
    lc = mc.LineCollection(lines, colors = "g",linewidths=2)
    axes.add_collection(lc)


# plt.figure(figsize=(8,8))
plt.xlim([0, 35])
plt.ylim([-5.0, 5.0])

x_list = state_data[:,0]
y_list = state_data[:,1]
yaw_list = state_data[:,2]

for x,y,yaw in zip(x_list,y_list,yaw_list):
    visualize([x,y,yaw])
plt.show()

In [ ]:
x_list